# Finding CHaracteristic function values for sample of transactions

## Key idea: Benford's Law


## Example: for subset {A,B}, the characteristic function value = sum over all invoices for price*quantity of subsets A, B, AB

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#reading the data
df = pd.read_excel('online_merged.xlsx', sheet_name='merged')

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590,United Kingdom
1,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590,United Kingdom
2,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590,United Kingdom
3,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590,United Kingdom
4,493414,21527,RETRO SPOT TRADITIONAL TEAPOT,12,2010-01-04 10:28:00,6.95,14590,United Kingdom


In [4]:

df.describe()
df['Description'] = df['Description'].str.strip()

In [5]:
df1 = df
df1.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,493414,21844,RETRO SPOT MUG,36,2010-01-04 10:28:00,2.55,14590,United Kingdom
1,493414,21533,RETRO SPOT LARGE MILK JUG,12,2010-01-04 10:28:00,4.25,14590,United Kingdom
2,493414,37508,NEW ENGLAND CERAMIC CAKE SERVER,2,2010-01-04 10:28:00,2.55,14590,United Kingdom
3,493414,35001G,HAND OPEN SHAPE GOLD,2,2010-01-04 10:28:00,4.25,14590,United Kingdom
4,493414,21527,RETRO SPOT TRADITIONAL TEAPOT,12,2010-01-04 10:28:00,6.95,14590,United Kingdom


In [6]:
#finding price * quantity as total price per item per invoice
df1_price = df1[(df1[['Price','Quantity']] > 0).all(axis=1)]
df1.describe()

,Invoice,Quantity,Price,Customer ID
count,770692.000000,770692.000000,770692.000000,770692.000000
mean,539271.351953,13.286099,2.993243,15327.973197
std,25509.968171,146.100973,11.160934,1696.651545
min,493414.000000,1.000000,0.001000,12346.000000
25%,517964.000000,2.000000,1.250000,13975.000000
50%,537868.000000,5.000000,1.950000,15260.000000
75%,562460.000000,12.000000,3.750000,16800.000000
max,581587.000000,80995.000000,8142.750000,18287.000000


In [7]:
# Checking for unique items

len(df.StockCode.unique())

4486

In [8]:
unq_desc = df.StockCode.unique()

In [9]:
unq_desc

array([21844, 21533, 37508, ..., 23562, 23561, 23843], dtype=object)

## Here, I have taken a sample of 5 distinct items and based on those items, i have samled the dataset. furthermore, from the sampled dataset, i have picked 50 random transactions to make the computation simpler. 

In [10]:
#np.random.seed(123)
sample_desc = np.random.choice(unq_desc, size = 7)

sample_txn = df1[df1['StockCode'].isin(sample_desc)]

sample_txn = sample_txn.sample(n = 100)

len(sample_txn.Invoice.unique())

sample_txn.groupby(['StockCode']).count()

sample_txn['TotalPrice'] = sample_txn['Price']*sample_txn['Quantity']

sample_txn

basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Invoice')) 
basket

Description,HANGING HEART JAR T-LIGHT HOLDER,POCKET BAG PINK PAISELY BROWN SPOT,SET OF 6 GIRLS CELEBRATION CANDLES,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 9 BLACK SKULL BALLOONS
Invoice,,,,,
494362,0.00,6.3,0.0,0.0,0.0
494787,7.50,0.0,0.0,0.0,0.0
497209,0.00,2.1,0.0,0.0,0.0
497360,0.00,0.0,15.0,0.0,0.0
497949,15.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...
575610,0.00,15.0,0.0,0.0,0.0
577055,38.16,0.0,0.0,0.0,0.0
577696,0.00,0.0,0.0,0.0,1.7


In [11]:
sample_txn.sort_values(['Invoice'])

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
5919,494362,22280,POCKET BAG PINK PAISELY BROWN SPOT,3,2010-01-13 15:56:00,2.10,17757,United Kingdom,6.30
9506,494787,84978,HANGING HEART JAR T-LIGHT HOLDER,6,2010-01-18 13:34:00,1.25,15164,United Kingdom,7.50
27199,497209,22280,POCKET BAG PINK PAISELY BROWN SPOT,1,2010-02-07 12:40:00,2.10,16788,United Kingdom,2.10
28491,497360,21126,SET OF 6 GIRLS CELEBRATION CANDLES,12,2010-02-09 09:17:00,1.25,12437,France,15.00
32792,497949,84978,HANGING HEART JAR T-LIGHT HOLDER,12,2010-02-15 12:10:00,1.25,16327,United Kingdom,15.00
...,...,...,...,...,...,...,...,...,...
708140,575610,22280,POCKET BAG PINK PAISELY BROWN SPOT,12,2011-11-10 12:45:00,1.25,16628,United Kingdom,15.00
724509,577055,84978,HANGING HEART JAR T-LIGHT HOLDER,36,2011-11-17 14:22:00,1.06,15189,United Kingdom,38.16
731592,577696,22437,SET OF 9 BLACK SKULL BALLOONS,2,2011-11-21 11:57:00,0.85,16406,United Kingdom,1.70
731902,577723,23309,SET OF 60 I LOVE LONDON CAKE CASES,2,2011-11-21 12:36:00,0.55,16713,United Kingdom,1.10


In [12]:
#finding average price of each item (for calculating shapley value)
avgs = sample_txn.groupby(['Description']).mean()
avgs

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
HANGING HEART JAR T-LIGHT HOLDER,537986.800000,21.145455,1.237455,15422.636364,23.842182
POCKET BAG PINK PAISELY BROWN SPOT,526247.428571,5.571429,1.614286,16279.714286,8.178571
SET OF 6 GIRLS CELEBRATION CANDLES,504241.800000,8.000000,1.250000,16065.200000,10.000000
SET OF 60 I LOVE LONDON CAKE CASES,564940.941176,23.352941,0.617059,15279.882353,11.091176
SET OF 9 BLACK SKULL BALLOONS,542474.000000,8.625000,0.850000,15646.125000,7.331250


In [13]:
# finding sum of quantities of each item (to calculate per item shapley value)
sums = sample_txn.groupby(['Description']).sum()
sums

,Invoice,Quantity,Price,Customer ID,TotalPrice
Description,,,,,
HANGING HEART JAR T-LIGHT HOLDER,29589274,1163,68.06,848245,1311.32
POCKET BAG PINK PAISELY BROWN SPOT,3683732,39,11.30,113958,57.25
SET OF 6 GIRLS CELEBRATION CANDLES,2521209,40,6.25,80326,50.00
SET OF 60 I LOVE LONDON CAKE CASES,9603996,397,10.49,259758,188.55
SET OF 9 BLACK SKULL BALLOONS,8679584,138,13.60,250338,117.30


In [14]:
backupbasket = basket

In [15]:
#adding few extra transactions for finding the combinations
basket.iloc[2]['FELT TOADSTOOL SMALL'] = 4.25
basket.iloc[5]['FELT TOADSTOOL SMALL'] = 4.25
basket.iloc[7]['SET OF 2 TEA TOWELS PING MICROWAVE'] = 11.80
basket.iloc[5]['DELUXE SEWING KIT'] = 11.90
basket.iloc[10]['IVORY CAFE HANGING LAMP'] = 31.2


In [16]:

basket

Description,HANGING HEART JAR T-LIGHT HOLDER,POCKET BAG PINK PAISELY BROWN SPOT,SET OF 6 GIRLS CELEBRATION CANDLES,SET OF 60 I LOVE LONDON CAKE CASES,SET OF 9 BLACK SKULL BALLOONS
Invoice,,,,,
494362,0.00,6.3,0.0,0.0,0.0
494787,7.50,0.0,0.0,0.0,0.0
497209,0.00,2.1,0.0,0.0,0.0
497360,0.00,0.0,15.0,0.0,0.0
497949,15.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...
575610,0.00,15.0,0.0,0.0,0.0
577055,38.16,0.0,0.0,0.0,0.0
577696,0.00,0.0,0.0,0.0,1.7


In [17]:
#Finding invoice wise sum

invoice_sum = list(basket.sum(axis = 1))


invoice_sum

[6.300000000000001,
 7.5,
 2.1,
 15.0,
 15.0,
 2.5,
 7.5,
 2.5,
 15.0,
 15.0,
 1.7,
 38.160000000000004,
 7.5,
 0.85,
 15.0,
 38.160000000000004,
 17.0,
 15.0,
 15.0,
 8.5,
 15.0,
 15.0,
 22.5,
 7.5,
 3.75,
 12.600000000000001,
 15.0,
 1.7,
 15.0,
 15.0,
 7.5,
 1.25,
 38.160000000000004,
 7.5,
 17.0,
 15.0,
 15.0,
 3.75,
 11.25,
 15.0,
 7.5,
 10.2,
 2.5,
 2.5,
 17.0,
 76.32000000000001,
 15.0,
 7.5,
 15.0,
 15.0,
 7.5,
 15.0,
 7.5,
 152.64000000000001,
 1.25,
 152.64000000000001,
 1.7,
 7.5,
 15.0,
 8.5,
 13.200000000000001,
 50.4,
 0.55,
 15.0,
 7.5,
 5.5,
 114.48,
 50.4,
 0.55,
 30.0,
 17.0,
 6.050000000000001,
 1.6500000000000001,
 15.0,
 7.5,
 7.5,
 1.1,
 2.2,
 15.0,
 0.55,
 7.5,
 2.46,
 15.0,
 1.25,
 1.7,
 38.160000000000004,
 7.5,
 0.85,
 8.5,
 13.200000000000001,
 26.400000000000002,
 1.25,
 15.0,
 3.4,
 114.48,
 15.0,
 38.160000000000004,
 1.7,
 1.1,
 13.200000000000001]

In [18]:
len(invoice_sum)

100

In [19]:
basket.to_csv ('basket.csv', index = True, header=True)

In [20]:
#finding distinct items to create subsets of items
list2 = list(basket.columns)

list2

['DOOR MAT NEIGHBOURHOOD WITCH',
 'DOORMAT NEIGHBOURHOOD WITCH',
 'MILK PAN PINK POLKADOT',
 'MILK PAN PINK RETROSPOT',
 'SET 2 PANTRY DESIGN TEA TOWELS',
 'WOODEN ROUNDERS GARDEN SET',
 'ZINC FOLKART SLEIGH BELLS']

In [19]:
print('itemwise sum: ',basket.sum())
print('total sum: ', basket.sum().sum())

itemwise sum:  Description
HANGING HEART JAR T-LIGHT HOLDER      1311.32
POCKET BAG PINK PAISELY BROWN SPOT      57.25
SET OF 6 GIRLS CELEBRATION CANDLES      50.00
SET OF 60 I LOVE LONDON CAKE CASES     188.55
SET OF 9 BLACK SKULL BALLOONS          117.30
dtype: float64
total sum:  1724.42


In [20]:
# Creating subsets

import itertools
subsets = []
stuff = list2
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        subsets.append(list(subset))

subsets
    

NameError: name 'list2' is not defined

In [ ]:
# finding the row wise items (have to optimise the step)

col_list_full = []
for index, row in basket.iterrows():
    col_list = []
    for col in basket:
            #if col in 
        if(basket.at[index,col]!=0.0):
            col_list.append(col)
    col_list_full.append(col_list)
    #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
col_list_full      

In [ ]:
# grouping the invoice and columns based on the subsets

before_grp = []
for i in range (len(basket)):
    temp = [str(col_list_full[i]),invoice_sum[i]]
    #print(col_list_full[i], ' ',invoice_sum[i])
    before_grp.append(temp)

before_grp

e_dataframe = pd.DataFrame(before_grp)
e_dataframe.columns = ['a', 'b']

e_dataframe

groupped_sum = pd.DataFrame(e_dataframe.groupby(['a']).sum())
groupped_sum.reset_index(level=0, inplace=True)
groupped_sum.columns = ['a', 'b']

In [ ]:
groupped_sum

In [ ]:
# finding characteristic function value for each group and assigning them to subset list
all_sums = []
for sub in subsets:
    sum1 = 0
    print('subset id:', subsets.index(sub))
    print("step1")
    #print(sub)
    print('------------')
    stuff = sub
    new_sub = []
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            new_sub.append(list(subset))
    print(new_sub)
    for item in new_sub:
        index = groupped_sum[groupped_sum['a'] == str(item)].index
        if index.empty:
            print('None')
        else:
            print(item, index)
            sum1+=groupped_sum['b'][index].values[0]
    all_sums.append(sum1)
    print('------------------------------------------------------------')

In [ ]:
all_sums

In [ ]:
new_dict = {}
for i in range(len(all_sums)):
    new_dict.update({str(subsets[i]) : all_sums[i] })
    print(subsets[i], " ", all_sums[i])

In [ ]:
def count_first_digit(data_str):
    mask=df[data_str]>1.
    data=list(df[mask][data_str])
    for i in range(len(data)):
        while data[i]>10:
            data[i]=data[i]/10
    first_digits=[int(x) for x in sorted(data)]
    unique=(set(first_digits))#a list with unique values of     first_digit list
    data_count=[]
    for i in unique:
        count=first_digits.count(i)
        data_count.append(count)
    total_count=sum(data_count)
    data_percentage=[(i/total_count)*100 for i in data_count]
    return  total_count,data_count, data_percentage# Benford's Law percentages for leading digits 1-9
#BENFORD = [30.1, 17.6, 12.5, 9.7, 7.9, 6.7, 5.8, 5.1, 4.6]

In [ ]:
import collections


BENFORD_PERCENTAGES = [0, 0.301, 0.176, 0.125, 0.097, 0.079, 0.067, 0.058, 0.051, 0.046]

def calculate(data):

    """
    Calculates a set of values from the numeric list
    input data showing how closely the first digits
    fit the Benford Distribution.
    Results are returned as a list of dictionaries.
    """

    results = []

    first_digits = list(map(lambda n: str(n)[0], data))
    first_digit_frequencies = collections.Counter(first_digits)

    for n in range(1, 10):

        data_frequency = first_digit_frequencies[str(n)]
        data_frequency_percent = data_frequency / len(data)
        benford_frequency = len(data) * BENFORD_PERCENTAGES[n]
        benford_frequency_percent = BENFORD_PERCENTAGES[n]
        difference_frequency = data_frequency - benford_frequency
        difference_frequency_percent = data_frequency_percent - benford_frequency_percent

        results.append({"n": n,
                        #"data_frequency":               data_frequency,
                        "data_frequency_percent":       data_frequency_percent,
                        #"benford_frequency":            benford_frequency,
                        "benford_frequency_percent":    benford_frequency_percent,
                        #"difference_frequency":         difference_frequency,
                        #"difference_frequency_percent": difference_frequency_percent
                       })

    return results 

In [ ]:
calculate(all_sums)

In [ ]:
import random

import benfordslaw


def main():

    print("-----------------")
    print("| codedrome.com |")
    print("| Benford's Law |")
    print("-----------------\n")

    data = get_random_data()
    #data = get_benford_data()

    print(len(data))

    benford_table = benfordslaw.calculate(data)
    print_as_table(benford_table)
    print()
    print_as_graph(benford_table)


def get_random_data():

    """
    Returns a list of 1000 numbers approximately
    following the uniform distribution NOT the
    Benford Distribution.
    """

    random_data = [0] * 1000

    random_data = list(map(lambda n: n + random.randint(1, 1000), random_data))

    return random_data


def get_benford_data():

    """
    Returns a list of about 1000 numbers
    approximately following the Benford Distribution.
    """

    benford_data = []

    for first_digit in range(1, 10):
        random_factor = random.uniform(0.8, 1.2)
        for num_count in range(1, int(1000 * benfordslaw.BENFORD_PERCENTAGES[first_digit] * random_factor)):
            start = first_digit * 1000
            benford_data.append(random.randint(start, start + 1000))

    return benford_data


def print_as_table(benford_table):

    width = 59

    print("-" * width)
    print("|   |      Data       |    Benford      |    Difference   |")
    print("| n |  Freq     Pct   |  Freq     Pct   |  Freq     Pct   |")
    print("-" * width)

    for item in benford_table:

        print("| {} | {:6.0f} | {:6.2f} | {:6.0f} | {:6.2f} | {:6.0f} | {:6.2f} |".format(item["n"],
                                   item["data_frequency"],
                                   item["data_frequency_percent"] * 100,
                                   item["benford_frequency"],
                                   item["benford_frequency_percent"] * 100,
                                   item["difference_frequency"],
                                   item["difference_frequency_percent"] * 100))

    print("-" * width)


def print_as_graph(benford_table):

    REDBG = "\x1B[41m"
    GREENBG = "\x1B[42m"
    RESET = "\x1B[0m"

    print("  0%       10%       20%       30%       40%       50%       60%       70%       80%       90%       100%")
    print("  |         |         |         |         |         |         |         |         |         |         |\n")

    for item in benford_table:

        print(" {} {}\n   {}\n  ".format(str(item["n"]),
                                         GREENBG + (" " * int(round(item["benford_frequency_percent"] * 100))) + RESET,
                                         REDBG + (" " * int(round(item["data_frequency_percent"] * 100))) + RESET))


main() 